In [1]:


%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') 

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)



Overwriting mapper.py


In [2]:


%%writefile reducer.py

import sys

current_key = None
word_sum = 0



Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [9]:


%%writefile mapper_rating.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') 

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
        
    print "%d\t%s" % (count, word)



Writing mapper_rating.py


In [8]:


%%writefile reducer_rating.py

import sys

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
        
    print "%s\t%d" % (word, count)



Writing reducer_rating.py


In [10]:
%%bash

OUT_DIR="wordcount"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null


OUT_DIR_2="word_rating"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount Rating" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D map.output.key.field.separator=\t \
    -D mapreduce.partition.keycomparator.options=-k1,1nr \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper_rating.py,reducer_rating.py \
    -mapper "python mapper_rating.py" \
    -reducer "python reducer_rating.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_2} > /dev/null

hdfs dfs -cat ${OUT_DIR_2}/part-00000 | head -7 | tail -1

is	126420


20/02/09 14:10:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/02/09 14:10:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/02/09 14:10:46 INFO mapred.FileInputFormat: Total input files to process : 1
20/02/09 14:10:46 INFO mapreduce.JobSubmitter: number of splits:2
20/02/09 14:10:46 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1581247117668_0012
20/02/09 14:10:46 INFO impl.YarnClientImpl: Submitted application application_1581247117668_0012
20/02/09 14:10:46 INFO mapreduce.Job: The url to track the job: http://46d11b27082c:8088/proxy/application_1581247117668_0012/
20/02/09 14:10:46 INFO mapreduce.Job: Running job: job_1581247117668_0012
20/02/09 14:10:52 INFO mapreduce.Job: Job job_1581247117668_0012 running in uber mode : false
20/02/09 14:10:52 INFO mapreduce.Job:  map 0% reduce 0%
20/02/09 14:11:11 INFO mapreduce.Job:  map 16% reduce 0%
20/02/09 14:11:17 INFO mapreduce.Job:  map 24% reduce 0%
20/02/09 14:11:23 INFO 